In [10]:
from helpers import *
import pandas as pd 
import numpy as np 
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app3", timeout=5)
import time

In [2]:
#Realizar Web Scraping 
"""
Region: Recomendable escribir nombre de region igual al configurado  por defecto en la pagina de portal inmobiliario
        fijarse en url de busqueda, por ejemplo: al escribir santiago en buscador principal, la pagina redirige a los resultados con la         categoria "santiago-metropolitana"
pagina principal: https://www.portalinmobiliario.com/
ejemplo nombre de region url: https://www.portalinmobiliario.com/arriendo/departamento/santiago-metropolitana

Pages: Cantidad de paginas de resultados para la busqueda de ofertas,  no se puede superar el numero de paginas de resultados que existen en portalinmobiliario según la region especificada.

Scopes: pueden ser arriendo o venta.

type: puede ser departamento, casa, sitio, parcela, oficina, comercial, industrial, agricola, terreno-en-construccion, bodega, estacionamiento,  loteo, otros 
(considerar que webscraping esta optimizado para departamentos, por lo que puede que al utilizar otro tipo de inmueble algunas columnas como baños, habitaciones,etc podrian no coincidir, debido a que la información para cada tipo de inmueble es distinta)
"""

df = webscraping_deptos(region="santiago-metropolitana",pages=1,type='departamento',scope='arriendo')
print(df)

Web Scraping Portal Inmobiliario
Buscando departamento para arriendo....
https://www.portalinmobiliario.com/arriendo/departamento/santiago-metropolitana/_Desde_1
https://www.portalinmobiliario.com/MLC-1372875297-santo-domingo-3251-departamento-1d1b-santiago-_JM#position=1&search_layout=stack&type=item&tracking_id=f527460a-397e-4da7-9bdb-1dce3653665f
Santo Domingo 3251, Santiago, Chile, Barrio Yungay, Santiago, RM (Metropolitana)
282.000
35 m² totales
1 dormitorio
1 baño
3
Departamentos en arriendo encontrados: 1
https://www.portalinmobiliario.com/MLC-1594148644-maule-150-departamento-2d1b-santiago-_JM#position=2&search_layout=stack&type=item&tracking_id=f527460a-397e-4da7-9bdb-1dce3653665f
Maule 150, Santiago, Chile, Bogotá - Sierra Bella, Santiago, RM (Metropolitana)
388.000
40 m² totales
2 dormitorios
1 baño
3
Departamentos en arriendo encontrados: 2
https://www.portalinmobiliario.com/MLC-1619702950-sara-del-campo-535-departamento-1d1b-santiago-_JM#position=3&search_layout=stack&type

In [3]:
from datetime import datetime

now = datetime.now()
formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")

df.to_csv('data-raw_'+formatted_time+'.csv',index=False)

In [4]:
def clean_data2(df):
    df['direccion'] = df['direccion'].str.replace("/",",")


    return df

In [5]:
df = clean_data2(df)

In [6]:
def clean_data(df):
    df = df[df.titulo.notnull()]
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['descripcion'] = df['descripcion'].str.replace(r"\n|\t","")
    df['ambiente'] = df['ambiente'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['comodidades'] = df['comodidades'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['otras caracteristicas'] = df['otras caracteristicas'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['precio'] = df['precio'].str.replace(".","")
    df['gastos comunes'] = df['gastos comunes'].str.replace("CLP","")
    df['habitaciones'] = df['habitaciones'].str.replace(r"-\s+\d\s\w*","").str.replace('Dormitorio',"").str.strip()
    df['banos'] = df['banos'].str.replace(r"-\s+\d\s\w*","").str.replace(r'Baño|s',"").str.strip()
    df['direccion'] = df['direccion'].str.replace("/",",")
    df['m2_utiles'] = df['m2_utiles'].str.replace(r"-.*$|m.","")

    return df

In [7]:
#df = clean_data(df)

In [8]:
df.to_csv('clean-data'+formatted_time+'.csv',index=False)

In [11]:
#Agregar coordenadas geograficas de direcciones a Data Frame con libreria geopy
targets_address= df['direccion'].tolist()

latitude = []
longitude = []

for target in targets_address:

    location = geolocator.geocode(target)
    if location == None:
        latitude.append(np.nan)
        longitude.append(np.nan)
    else:
        latitude.append(location.latitude)
        longitude.append(location.longitude)

In [12]:
#Almacenar resultados en Data Frame
df['latitud'] = latitude
df['longitud'] = longitude

In [13]:
#Crear nuevo data frame con  direcciones validas y coordeandas geograficas no nulas 
geo_df = df[~df['latitud'].isnull()]
geo_df['ciudad'] = 'Santiago'

/tmp/ipykernel_45108/2314933894.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df['City'] = 'Santiago'


In [14]:
geo_df.to_csv('santiago'+formatted_time+'.csv',index=False)